<a href="https://colab.research.google.com/github/Sitraka17/Learning-Data-Science/blob/main/RL_minicourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup requirements and import dependencies

In [ ]:
!pip install torch ray[rllib]==1.12.1 gym Box2D matplotlib tensorboard redis gym-super-mario-bros pickle5
!test -e utils.py && rm utils.py
!wget "https://matthieu-zimmer.net/~matthieu/courses/rl-minicourse/colab/utils.py"

import torch # NN framework
import gym  # RL environment
import ray  # RL algorithms
import time
import utils
from ray.rllib.agents.ppo import PPOTrainer

# 2. Interaction Loop: Exercise 1

Interact with an environment for one complete trajectory by playing random actions and print the sum of reward obtained

# 3. Interaction Loop: Exercise 2
With the help of print and the display of the environment, guess if the actions are continuous or discrete (how many possible action if discrete), what the meaning of the action is, what the state is and what the reward function is.

# 4. Training
Train a policy with the PPO algorithm for 10 iterations.

# 5. Monitoring Training 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results

# 6. Saving and Loading a policy

# 7. Evaluation: Exercise 3

Load and evaluate the policy you trained before. To do so, you can reuse code from Exercise 1, but instead of playing random action, use the new policy.
Instead of evaluating the policy on one trajectory, use an average over multiples trajectories.

# 8. Hyperparameters Optimisation: Exercise 4

Try obtaining higher rewards by retraining with other hyperparameters. 

In [ ]:
# default values of some hyperparemeters of PPO
default_config = {
    "framework": "torch",
    "num_gpus": 0,
    "num_workers": 1,
    "num_cpus_per_worker": 1,
    "num_envs_per_worker": 1,
    "gamma": 0.99,
    "lr": 0.0001,
    "train_batch_size": 200,
}

# 9. Customised Environment: Exercise 5
By looking at the simple Shower environment (defined below), create your own new environment to reproduce the following MDP: https://en.wikipedia.org/wiki/File:Markov_Decision_Process.svg


In [ ]:
import numpy as np
from ray.tune import register_env

class ShowerEnv(gym.Env):
  def __init__(self):
    # 3 discrete actions
    self.action_space = gym.spaces.Discrete(3)
    # single continuous scalar to describe the state
    self.observation_space = gym.spaces.Box(low=0, high=75, shape=(1,))

  def reset(self):
    # initial temperature between 33° and 43°
    self.temperature = np.array([38 + np.random.randint(-5, 5)]).astype(float)
    # number of step in one shower
    self.shower_length = 10
  
  def step(self, action):
    # the new temperature depends on the action and some random values
    self.temperature += np.random.randint(-3, 2) + action

    # decrease length
    self.shower_length -= 1

    # stop the episode once the length is 0
    done = self.shower_length <= 0

    # compute reward
    if self.temperature >= 37 and self.temperature <= 39:
      reward = 1
    else:
      reward = -1

    return self.temperature, reward, done, []

# register the environment with rllib
register_env('MyShowerEnv-v0', lambda x: ShowerEnv())
# instead of using gym.make use the following
env = ShowerEnv()

env.reset()
env.step(1)


(array([37.]), 1, False, [])

# 10. Try other algorithms: Exercise 6
Rllib features numerous algorithms. Visit https://docs.ray.io/en/latest/rllib/rllib-algorithms.html, pick one that suits your needs and use it to train an agent on your customis
ed environment. 